In [ ]:
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/Colab Notebooks/test_image/'

Mounted at /content/drive


In [ ]:
# 전역 변수 설정
TOTAL_IMAGES = 19
WIDTH, HEIGHT = 512, 512
K = [4, 16, 64]
N = [4, 8, 16, 32, 64, 128]

In [ ]:
# 이미지 파일 이름 가져오기
image_data = {}
for i in range(TOTAL_IMAGES):  # I0 ~ I18
    filename = f'I{i}.img'
    filepath = f'{PATH}{filename}'

    try:
        with open(filepath, 'rb') as file:
            data = file.read()
            image_data[i] = data
        print(f'{i} 파일 읽기 성공')
    except FileNotFoundError:
        print(f'{i} 파일 읽기 실패')

0 파일 읽기 성공
1 파일 읽기 성공
2 파일 읽기 성공
3 파일 읽기 성공
4 파일 읽기 성공
5 파일 읽기 성공
6 파일 읽기 성공
7 파일 읽기 성공
8 파일 읽기 성공
9 파일 읽기 성공
10 파일 읽기 성공
11 파일 읽기 성공
12 파일 읽기 성공
13 파일 읽기 성공
14 파일 읽기 성공
15 파일 읽기 성공
16 파일 읽기 성공
17 파일 읽기 성공
18 파일 읽기 성공


In [ ]:
# 이미지 K차원 변환
def reshape_image(data, n):

    dimension_size = int(n ** 0.5)
    result = []

    dimension_width = WIDTH // dimension_size
    dimension_height = HEIGHT // dimension_size
    for d in range(TOTAL_IMAGES):
        for block_y in range(dimension_height):
            for block_x in range(dimension_width):
                block = []
                for y in range(dimension_size):
                    for x in range(dimension_size):
                        pixel_x = block_x*dimension_size + x
                        pixel_y = block_y*dimension_size + y
                        index = pixel_y*WIDTH + pixel_x
                        block.append(data[d][index])
                result.append(block)
    return result

In [ ]:
# k-means 클러스트링 함수
import random

class k_means():
    def __init__(self, max_iter=100):
        self.max_iter = max_iter

    def initialize_centroids(self, dataset, clu, prev_centroids=None):
        if prev_centroids is None:
            return [random.choice(dataset) for _ in range(clu)]
        else:
            new_centroids = prev_centroids.copy()
            while len(new_centroids) < clu:
                new_centroids.append(random.choice(dataset))
            return new_centroids[:clu]

    def cal_distance(self, a, b):
        return sum((a_i - b_i) ** 2 for a_i, b_i in zip(a, b)) ** 0.5

    def assign_clusters(self, centroids, dataset):
        clusters = [[] for _ in range(len(centroids))]
        for point in dataset:
            distances = [self.cal_distance(point, centroid) for centroid in centroids]
            index = distances.index(min(distances))
            clusters[index].append(point)
        return clusters

    def cal_new_centroids(self, clusters, dataset):
        new_centroids = []
        for cluster in clusters:
            if cluster:
                avr_centroid = [round(sum(dim)/len(dim), 4) for dim in zip(*cluster)]
                new_centroids.append(avr_centroid)
            else:
                new_centroids.append(random.choice(dataset))
        return new_centroids

    def execute_k_means(self, dataset, clu, prev_centroids=None):
        centroids = self.initialize_centroids(dataset, clu, prev_centroids)
        for _ in range(self.max_iter):
            clusters = self.assign_clusters(centroids, dataset)
            new_centroids = self.cal_new_centroids(clusters, dataset)
            if centroids == new_centroids:
                break
            centroids = new_centroids
        return centroids

# 파일 저장 함수
def save_centroids(centroids, n, dim):
    filename = f'/{dim}_{n}_centroids.txt'
    filepath = f'{PATH}{filename}'
    with open(filepath, 'w') as file:
        for row in centroids:
            line = ','.join(map(str, row))
            file.write(line + '\n')

def run_k_means(dataset, N, dim):
    kmeans = k_means()
    final_results = {}
    prev_centroids = None
    for n in N:
        centroids = kmeans.execute_k_means(dataset, n, prev_centroids)
        final_results[n] = centroids
        prev_centroids = centroids
        save_centroids(centroids, n, dim)
        print(n, centroids)
    return final_results

In [ ]:
for k in K:
    reshap_images = reshape_image(image_data, k)
    result = run_k_means(reshap_images, N, k)

KeyboardInterrupt: 